# Fit a linear trend to annual minimum AARs for all sites

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

figures_out_path = os.path.join(base_path, 'figures')

In [3]:
# -----Load and compile snowlines
snowlines_path = os.path.join(scm_path, 'all_snowlines')
snowlines_fn = 'all_snowlines.csv'
# check if snowlines path exists
if not os.path.exists(snowlines_path):
    os.mkdir(snowlines_path)
# check if all snowlines CSV exists
if not os.path.exists(os.path.join(snowlines_path, snowlines_fn)):
    # compile all RGI glacier boundaries
    snowlines = pd.DataFrame()
    for site_name in tqdm(site_names):
        snowline_path = os.path.join(study_sites_path, site_name)
        snowline_fns = glob.glob(os.path.join(snowline_path, '*_snowlines.csv'))
        if len(snowline_fns) > 0:
            snowline_fn = snowline_fns[0]
            snowline = pd.read_csv(snowline_fn)
            snowlines = pd.concat([snowlines, snowline])
    snowlines.reset_index(drop=True, inplace=True)
    snowlines.to_csv(os.path.join(snowlines_path, snowlines_fn), index=False)
    print('All snowlines saved to file: ', os.path.join(snowlines_path, snowlines_fn))

else:
    # load from file if it already exists
    snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))
    snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
    print('All snowlines loaded from file.')

snowlines

All snowlines loaded from file.


/var/folders/9h/sg2rw9f10lq5jmfch9hnj6gc0000gq/T/ipykernel_29856/2315288995.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))


,site_name,datetime,snowlines_coords_X,snowlines_coords_Y,HorizontalCRS,VerticalCRS,snowline_elevs_m,snowline_elevs_median_m,SCA_m2,AAR,ELA_from_AAR_m,dataset,geometry,CRS,HorizontalReference,VerticalReference
0,RGI60-01.00032,2013-05-26 15:08:53,[],[],EPSG:32606,EGM96 geoid (EPSG:5773),1403.1963,1403.1963,10233000.0,0.970302,1447.7217,Landsat,[],NaN,NaN,NaN
1,RGI60-01.00032,2013-10-26 15:02:47,"[498135.0, 498120.0, 498105.0, 498075.0, 49804...","[7061070.0, 7061085.0, 7061100.0, 7061100.0, 7...",EPSG:32606,EGM96 geoid (EPSG:5773),"[1436.056635197854, 1436.056635197854, 1433.88...",1438.7190,10012500.0,0.974168,1438.1063,Landsat,"LINESTRING (498135 7061070, 498120 7061085, 49...",NaN,NaN,NaN
2,RGI60-01.00032,2014-06-07 15:00:43,"[499275.0, 499260.0, 499245.0, 499230.0, 49921...","[7060140.0, 7060155.0, 7060170.0, 7060155.0, 7...",EPSG:32606,EGM96 geoid (EPSG:5773),"[1565.076636493261, 1565.076636493261, 1561.70...",1541.0497,12474000.0,0.933774,1519.7489,Landsat,"LINESTRING (499275 7060140, 499260 7060155, 49...",NaN,NaN,NaN
3,RGI60-01.00032,2014-10-13 15:01:17,"[503535.0, 503505.0, 503475.0, 503445.0, 50343...","[7057650.0, 7057650.0, 7057650.0, 7057650.0, 7...",EPSG:32606,EGM96 geoid (EPSG:5773),"[2031.2784724049204, 2029.1873066596477, 2027....",2031.2784,8817300.0,0.979210,1427.6902,Landsat,"LINESTRING (503535 7057650, 503505 7057650, 50...",NaN,NaN,NaN
4,RGI60-01.00032,2015-10-23 15:25:39,[],[],EPSG:32606,EGM96 geoid (EPSG:5773),1402.4318,1402.4318,10414800.0,0.999693,1404.9034,Sentinel-2_TOA,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52350,RGI60-02.17741,2023-10-27 19:20:56,[],[],EPSG:32610,EGM96 geoid (EPSG:5773),1406.2806,1406.2806,5814500.0,0.989079,1484.3608,Sentinel-2_SR,[],NaN,NaN,NaN
52351,RGI60-02.17741,2023-10-27 19:20:56,[],[],EPSG:32610,EGM96 geoid (EPSG:5773),1406.2806,1406.2806,5675400.0,0.976228,1550.5474,Sentinel-2_TOA,[],NaN,NaN,NaN
52352,RGI60-02.17741,2023-10-27 19:20:56,[],[],EPSG:32610,EGM96 geoid (EPSG:5773),1406.2806,1406.2806,5675400.0,0.976228,1550.5474,Sentinel-2_TOA,[],NaN,NaN,NaN
52353,RGI60-02.17741,2023-10-29 19:10:56,[],[],EPSG:32610,EGM96 geoid (EPSG:5773),1406.2806,1406.2806,6038900.0,0.960446,1630.0000,Sentinel-2_SR,[],NaN,NaN,NaN


In [23]:
# -----For each site, identify minimum AARs, fit linear trendlines
# add year column
snowlines['Year'] = snowlines['datetime'].dt.isocalendar().year
# subset to dates after 2015
snowlines_subset = snowlines.loc[snowlines['Year'] > 2016]
# initialize dataframe for storing minimum AARs and linear fits
min_aars_df = pd.DataFrame()
# iterate over site names
for site_name in tqdm(snowlines['site_name'].drop_duplicates().values):
    # subset snowlines to site
    snowlines_site = snowlines.loc[snowlines['site_name']==site_name]
    # extract minimum AARs and dates
    min_aars = snowlines_site.groupby(['Year'])['AAR'].min()
    min_dts = []
    # iterate over years to extract dates
    for year, min_aar in zip(np.array(min_aars.index), min_aars.values):
        min_dt = snowlines_site.loc[(snowlines_site['Year']==year) & (snowlines_site['AAR']==min_aar)]['datetime'].values[0]
        min_dts.append(min_dt)
    # fit a linear model to dates and AARs
    model = LinearRegression()
    model_fit = model.fit(np.arange(0, len(min_aars)).reshape(-1, 1), min_aars)

    # save in dataframe
    min_aar_df = pd.DataFrame({'site_name': [site_name],
                               'minimum_AARs': [min_aars.values],
                               'minimum_AARs_dts': [min_dts],
                               'linear_fit_coef': model_fit.coef_,
                               'linear_fit_intercept': [model_fit.intercept_]
                              })
    # concatenate to full dataframe
    min_aars_df = pd.concat([min_aars_df, min_aar_df])

# save to file
min_aars_fn = 'minimum_AARs_linear_fit.csv'
min_aars_df.to_csv(os.path.join(snowlines_path, min_aars_fn), index=False)
print('Data table saved to file: ', os.path.join(snowlines_path, min_aars_fn))
min_aars_df.reset_index(drop=True, inplace=True)
min_aars_df

  0%|          | 0/180 [00:00<?, ?it/s]

/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureW

Data table saved to file:  /Users/raineyaberle/Research/PhD/snow_cover_mapping/all_snowlines/minimum_AARs_linear_fit.csv


/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureW

,site_name,minimum_AARs,minimum_AARs_dts,linear_fit_coef,linear_fit_intercept
0,RGI60-01.00032,"[0.9703021, 0.9337735, 0.99969286, 0.3534108, ...","[2013-05-26T15:08:53.000000000, 2014-06-07T15:...",-0.092355,0.849552
1,RGI60-01.00033,"[0.08487283, 0.5816994, 0.16841005, 0.01180021...","[2013-08-07T15:03:04.000000000, 2014-06-07T15:...",-0.030753,0.244175
2,RGI60-01.00037,"[0.9760566, 0.9872925, 0.95119995, 0.856186, 0...","[2013-10-26T15:02:47.000000000, 2014-10-13T15:...",-0.064108,0.917300
3,RGI60-01.00038,"[0.5295494, 0.64329374, 0.36518896, 0.29750726...","[2013-09-15T15:08:48.000000000, 2014-09-11T15:...",-0.021799,0.488275
4,RGI60-01.00046,"[0.9972489, 0.84014374, 0.6247682, 0.3659833, ...","[2013-05-26T15:09:17.000000000, 2014-09-25T15:...",-0.051417,0.743347
...,...,...,...,...,...
175,RGI60-02.17029,"[1.0, 0.94189316, 0.8429764, 0.94329226, 0.883...","[2013-10-04T12:26:14.000000000, 2014-09-21T12:...",-0.105659,1.100049
176,RGI60-02.17736,"[0.58455163, 0.09070016]","[2021-08-18T13:20:54.000000000, 2023-09-17T13:...",-0.493851,0.584552
177,RGI60-02.17738,"[0.70348215, 0.7522158, 0.67661315, 0.7501371,...","[2013-09-12T13:03:25.000000000, 2014-09-15T13:...",-0.059709,0.817119
178,RGI60-02.17739,"[0.58455163, 0.20869294]","[2021-08-18T13:20:54.000000000, 2023-09-09T13:...",-0.375859,0.584552
